### Introduction to machine learning with TensorFlow

This notebook is intended to be run in a local Python environment with access to the TensorFlow package.

Package imports:

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

Read in data - we are using the abalone mollusc dataset from the UCI ML dataset repository:

In [2]:
spam_data = fetch_ucirepo(id=1)

spam_data.data.original.head(5)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Using the UCI ML repo package, the dataset comes with the variables already split into features and labels, which we can read into separate DataFrames:

In [3]:
# Dataset features, X
spam_features = spam_data.data.features

# Dataset labels, y
spam_labels = spam_data.data.targets

Preview features:

In [4]:
spam_features.head(5)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


Preview labels:

In [5]:
spam_labels.head(5)

,Rings
0,15
1,7
2,9
3,10
4,7


The goal of our machine learning model is to predict the age of an abalone by its physical characteristics. We will use functions defined in [tensorflow_functions.py](modules/tensorflow_functions.py) to build and train a linear regression model.

In [6]:
from modules.tensorflow_functions import *